<center><u><H1>YOLO -- Darkflow -- Python</H1></u></center>

### Import Libraries

In [ ]:
import os
import cv2
import tensorflow as tf
from darkflow.net.build import TFNet
from icrawler.builtin import GoogleImageCrawler

## Download Images

In [ ]:
for keyword in ['car', 'motorcycle']: # Name image classes you want to download from Google in the array
    
    google_crawler = GoogleImageCrawler(
        parser_threads=2,
        downloader_threads=4,
        storage={'root_dir': 'images/{}'.format(keyword)}
    
    )
    google_crawler.crawl(
        keyword=keyword, max_num=200, min_size=(200, 200))

## Rename Images

In [ ]:
n = 0
image_dir = '/train/images'

for image in os.scandir(image_dir): 
    os.rename(image.path, os.path.join(image_dir, '{:01}.jpg'.format(n)))
    n += 1

### Configure Tensorflow 

In [ ]:
# Config TF, set True if using GPU
config = tf.ConfigProto(log_device_placement = False)
config.gpu_options.allow_growth = False 

### Train 

In [ ]:
with tf.Session(config=config) as sess:
    options = {
            'model': 'cfg/yolo_1c.cfg',
            'load': 'bin/yolo.weights',
            'threshold': 0.1,
            'batch': 5,
            'epoch': 500,
            'train': True,
            'annotations': 'train/annotations',
            'dataset': 'train/images'
            #'gpu': 1.0 # uncomment these if using GPU
               }
    tfnet = TFNet(options)  

### Load Trained Model

In [ ]:
with tf.Session(config=config) as sess:
    options = {
            'model': 'cfg/yolo_1c.cfg',
            'load': 3500,
            'threshold': 0.5,
            #'gpu': 1.0 # uncomment these if using GPU
               }
    tfnet = TFNet(options)    

### Test Trained Model

In [ ]:
img = cv2.imread('./train/images/0024.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
results = tfnet.return_predict(img)
print(results)

### Display Results

In [ ]:
img = cv2.imread('./train/images/0024.jpg')
for (i, result) in enumerate(results):
    x = result['topleft']['x']
    w = result['bottomright']['x']-result['topleft']['x']
    y = result['topleft']['y']
    h = result['bottomright']['y']-result['topleft']['y']
    cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
    label_position = (x + int(w/2)), abs(y - 10)
    cv2.putText(img, result['label'], label_position , cv2.FONT_HERSHEY_SIMPLEX,0.5, (255,255,255), 2)

cv2.imshow("YOLO", img)
cv2.waitKey(0)
cv2.destroyAllWindows()